In [7]:
# Import libaries
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import creds
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from recommender_v2 import get_user_track
from recommender_v2 import cosine_recs

In [4]:
df = pd.read_csv('drake_songs_dataset.csv')
df.head()

,track_uri,track_name,album_name,duration_ms,danceability,energy,key,loudness,speechiness,acousticness,...,liveness,valence,tempo,mode,type,id,uri,track_href,analysis_url,time_signature
0,spotify:track:67nepsnrcZkowTxMWigSbb,MELTDOWN (feat. Drake),UTOPIA,246134,0.557,0.774,7,-5.275,0.3510,0.012000,...,0.396,0.397,111.975,0,audio_features,67nepsnrcZkowTxMWigSbb,spotify:track:67nepsnrcZkowTxMWigSbb,https://api.spotify.com/v1/tracks/67nepsnrcZko...,https://api.spotify.com/v1/audio-analysis/67ne...,4
1,spotify:track:6DCZcSspjsKoFjzjrWoCdn,God’s Plan,Scorpion,198973,0.754,0.449,7,-9.211,0.1090,0.033200,...,0.552,0.357,77.169,1,audio_features,6DCZcSspjsKoFjzjrWoCdn,spotify:track:6DCZcSspjsKoFjzjrWoCdn,https://api.spotify.com/v1/tracks/6DCZcSspjsKo...,https://api.spotify.com/v1/audio-analysis/6DCZ...,4
2,spotify:track:3F5CgOj3wFlRv51JsHbxhe,Jimmy Cooks (feat. 21 Savage),"Honestly, Nevermind",218365,0.529,0.673,0,-4.711,0.1750,0.000307,...,0.093,0.366,165.921,1,audio_features,3F5CgOj3wFlRv51JsHbxhe,spotify:track:3F5CgOj3wFlRv51JsHbxhe,https://api.spotify.com/v1/tracks/3F5CgOj3wFlR...,https://api.spotify.com/v1/audio-analysis/3F5C...,4
3,spotify:track:2G7V7zsVDxg1yRsu7Ew9RJ,In My Feelings,Scorpion,217925,0.835,0.626,1,-5.833,0.1250,0.058900,...,0.396,0.350,91.030,1,audio_features,2G7V7zsVDxg1yRsu7Ew9RJ,spotify:track:2G7V7zsVDxg1yRsu7Ew9RJ,https://api.spotify.com/v1/tracks/2G7V7zsVDxg1...,https://api.spotify.com/v1/audio-analysis/2G7V...,4
4,spotify:track:7aRCf5cLOFN1U7kvtChY1G,Search & Rescue,Search & Rescue,272113,0.817,0.440,10,-8.482,0.0734,0.060300,...,0.330,0.544,142.024,0,audio_features,7aRCf5cLOFN1U7kvtChY1G,spotify:track:7aRCf5cLOFN1U7kvtChY1G,https://api.spotify.com/v1/tracks/7aRCf5cLOFN1...,https://api.spotify.com/v1/audio-analysis/7aRC...,4


In [10]:
features = [
        'danceability', 'energy', 'key', 'loudness',
        'speechiness', 'acousticness', 'instrumentalness', 
        'liveness', 'valence', 'tempo'
    ]

In [81]:
def test_recs(df, features, model):
    # Randomly sample songs that will be input songs
    
    sample = df.sample(n = 5, replace = False, random_state = 42)

    # Create output songs 
    recs = pd.DataFrame(columns = features)

    # Iterate through input songs and generate predictions
    for track in sample.iterrows():
        recs = pd.concat([recs, model(df, features, 5, track[1]['track_name'])])



In [58]:
sample = df.sample(n = 5, replace = False, random_state = 42)


In [82]:
a = test_recs(df, features, cosine_recs)
a

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,...,track_name,album_name,duration_ms,mode,type,id,uri,track_href,analysis_url,time_signature
405,0.720,0.649,5,-5.187,0.2390,0.3170,0.000000,0.0886,0.2430,147.941,...,"Deuces Remix (feat. Drake, Kanye West & André ...",Deuces Remix,338920.0,0.0,audio_features,4QKMuKPhCK8Aji8MaeJL4T,spotify:track:4QKMuKPhCK8Aji8MaeJL4T,https://api.spotify.com/v1/tracks/4QKMuKPhCK8A...,https://api.spotify.com/v1/audio-analysis/4QKM...,4.0
62,0.746,0.687,1,-7.475,0.3820,0.1550,0.000000,0.1370,0.2830,143.931,...,Trophies,Rise Of An Empire (Deluxe Edition),188320.0,1.0,audio_features,6HfOzLLjsaXsehIFEsrxTk,spotify:track:6HfOzLLjsaXsehIFEsrxTk,https://api.spotify.com/v1/tracks/6HfOzLLjsaXs...,https://api.spotify.com/v1/audio-analysis/6HfO...,4.0
336,0.748,0.766,1,-5.110,0.2560,0.3380,0.000000,0.0861,0.5010,141.840,...,Schemin Up (feat. Drake and P. Reign),Schemin Up (feat. Drake and P. Reign),330770.0,1.0,audio_features,6wBpgsERq6XpJfIjWXOrnn,spotify:track:6wBpgsERq6XpJfIjWXOrnn,https://api.spotify.com/v1/tracks/6wBpgsERq6Xp...,https://api.spotify.com/v1/audio-analysis/6wBp...,4.0
172,0.414,0.677,1,-5.120,0.2540,0.0248,0.000000,0.0793,0.0853,172.521,...,Emotionless,Scorpion,302173.0,1.0,audio_features,5Psnhdkyanjpgc2P8A5TSM,spotify:track:5Psnhdkyanjpgc2P8A5TSM,https://api.spotify.com/v1/tracks/5Psnhdkyanjp...,https://api.spotify.com/v1/audio-analysis/5Psn...,4.0
57,0.727,0.599,1,-6.941,0.3270,0.3170,0.000000,0.2190,0.3860,167.909,...,R.I.C.O. (feat. Drake),Dreams Worth More Than Money,197133.0,1.0,audio_features,0Y0TOsE1q11qgbi7c5WZsG,spotify:track:0Y0TOsE1q11qgbi7c5WZsG,https://api.spotify.com/v1/tracks/0Y0TOsE1q11q...,https://api.spotify.com/v1/audio-analysis/0Y0T...,3.0
163,0.643,0.178,4,-8.560,0.0588,0.4840,0.010800,0.1220,0.0674,85.042,...,Can I,Care Package,189289.0,0.0,audio_features,3e0ZGE7Gp034iLknjQk4QW,spotify:track:3e0ZGE7Gp034iLknjQk4QW,https://api.spotify.com/v1/tracks/3e0ZGE7Gp034...,https://api.spotify.com/v1/audio-analysis/3e0Z...,4.0
48,0.697,0.235,7,-8.651,0.0534,0.6200,0.000144,0.1070,0.3780,99.947,...,Jungle,If You're Reading This It's Too Late,320400.0,1.0,audio_features,7JXZq0JgG2zTrSOAgY8VMC,spotify:track:7JXZq0JgG2zTrSOAgY8VMC,https://api.spotify.com/v1/tracks/7JXZq0JgG2zT...,https://api.spotify.com/v1/audio-analysis/7JXZ...,4.0
392,0.654,0.383,3,-9.875,0.0363,0.1290,0.000755,0.1030,0.2680,139.950,...,The Zone - Original,Thursday (Original),418057.0,0.0,audio_features,53qYItjefG5SUf62428dIw,spotify:track:53qYItjefG5SUf62428dIw,https://api.spotify.com/v1/tracks/53qYItjefG5S...,https://api.spotify.com/v1/audio-analysis/53qY...,4.0
148,0.694,0.314,0,-7.820,0.1160,0.6110,0.000002,0.1170,0.3410,119.905,...,Redemption,Views,333947.0,1.0,audio_features,4cRBqWBjuccCowYVHFlXK6,spotify:track:4cRBqWBjuccCowYVHFlXK6,https://api.spotify.com/v1/tracks/4cRBqWBjuccC...,https://api.spotify.com/v1/audio-analysis/4cRB...,4.0
297,0.593,0.393,1,-11.736,0.0827,0.3940,0.000003,0.1100,0.1930,136.866,...,Fall For Your Type (feat. Drake),Best Night Of My Life,269533.0,1.0,audio_features,0hRPRbA2B1CYmGNhWhFjwF,spotify:track:0hRPRbA2B1CYmGNhWhFjwF,https://api.spotify.com/v1/tracks/0hRPRbA2B1CY...,https://api.spotify.com/v1/audio-analysis/0hRP...,4.0


,track_uri,track_name,album_name,duration_ms,danceability,energy,key,loudness,speechiness,acousticness,...,liveness,valence,tempo,mode,type,id,uri,track_href,analysis_url,time_signature
163,spotify:track:3e0ZGE7Gp034iLknjQk4QW,Can I,Care Package,189289,0.643,0.178,4,-8.560,0.0588,0.484,...,0.122,0.0674,85.042,0,audio_features,3e0ZGE7Gp034iLknjQk4QW,spotify:track:3e0ZGE7Gp034iLknjQk4QW,https://api.spotify.com/v1/tracks/3e0ZGE7Gp034...,https://api.spotify.com/v1/audio-analysis/3e0Z...,4
48,spotify:track:7JXZq0JgG2zTrSOAgY8VMC,Jungle,If You're Reading This It's Too Late,320400,0.697,0.235,7,-8.651,0.0534,0.620,...,0.107,0.3780,99.947,1,audio_features,7JXZq0JgG2zTrSOAgY8VMC,spotify:track:7JXZq0JgG2zTrSOAgY8VMC,https://api.spotify.com/v1/tracks/7JXZq0JgG2zT...,https://api.spotify.com/v1/audio-analysis/7JXZ...,4
392,spotify:track:53qYItjefG5SUf62428dIw,The Zone - Original,Thursday (Original),418057,0.654,0.383,3,-9.875,0.0363,0.129,...,0.103,0.2680,139.950,0,audio_features,53qYItjefG5SUf62428dIw,spotify:track:53qYItjefG5SUf62428dIw,https://api.spotify.com/v1/tracks/53qYItjefG5S...,https://api.spotify.com/v1/audio-analysis/53qY...,4
148,spotify:track:4cRBqWBjuccCowYVHFlXK6,Redemption,Views,333947,0.694,0.314,0,-7.820,0.1160,0.611,...,0.117,0.3410,119.905,1,audio_features,4cRBqWBjuccCowYVHFlXK6,spotify:track:4cRBqWBjuccCowYVHFlXK6,https://api.spotify.com/v1/tracks/4cRBqWBjuccC...,https://api.spotify.com/v1/audio-analysis/4cRB...,4
297,spotify:track:0hRPRbA2B1CYmGNhWhFjwF,Fall For Your Type (feat. Drake),Best Night Of My Life,269533,0.593,0.393,1,-11.736,0.0827,0.394,...,0.110,0.1930,136.866,1,audio_features,0hRPRbA2B1CYmGNhWhFjwF,spotify:track:0hRPRbA2B1CYmGNhWhFjwF,https://api.spotify.com/v1/tracks/0hRPRbA2B1CY...,https://api.spotify.com/v1/audio-analysis/0hRP...,4
